# Agriculture Demand Model

This is the Agriculture energy demand model. The input data files are from EGEDA and the macro model. The model uses Prophet to generate predictions for each economy and fuel. 

You can adjust the fuels you are predicting.

You can also add or remove 'features', e.g., GDP, population, GDP per capita, etc.

In [ ]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd
from fbprophet import Prophet

## 1. Import data and process the data

In [ ]:
# note: the EGEDA data has been cleaned

df_EGEDA = pd.read_csv("../data/final/EGEDA_2019_09_25_tidy.csv",
                      index_col=['Economy'])

In [ ]:
df_EGEDA.head()

In [ ]:
# This shows what demand series are available.
# Agriculture is '15.2 Agriculture'

df_EGEDA.columns.values.tolist()

In [ ]:
# select only the demand you are interested in, e.g., residential

df_historical = df_EGEDA[['Year','Fuel Code','15.2 Agriculture']]
df_historical.head()

In [ ]:
df_historical.tail()

In [ ]:
# we want to know the fuels

df_historical['Fuel Code'].unique()

In [ ]:
# putting in to a table format to make it easy to select only the fuel columns we are interested in

res_table = pd.pivot_table(df_historical, values='15.2 Agriculture', index=['Economy', 'Year'], columns=['Fuel Code'], aggfunc=np.sum)

In [ ]:
# you can add or remove the fuels you want to model

targets = ['Coal','Elec','Gas','Heat','Nuc','Oil','Oth','PetP', 'TotRen','Tot']
targets_tuple = tuple(targets)
df_res = res_table[targets]
df_res

In [ ]:
# this is the historical macro data

df_macro = pd.read_csv('../data/raw/Industry/MacroHistorical.csv',
                       index_col=['Economy','Year'])
df_macro

In [ ]:
# combine the EGEDA data and macro data

df_hist = pd.merge(df_res,df_macro,how='left',on=['Economy','Year'])

# BIG ASSUMPTION in the next cell

In [ ]:
# there are missing data
# some of these might be missing or they might mean they are zero

           
#df_hist = df_hist.fillna(method='backfill')
#df_hist = df_hist.fillna(method='ffill')

df_hist = (df_hist.dropna(how="all", axis=1)
        .dropna(how="any", axis=0))

In [ ]:
df_hist.head()

In [ ]:
df_hist.tail()

In [ ]:
# add the YYYY-MM-12 column format for Prophet

df_hist = df_hist.reset_index(level='Year')
df_hist['ds'] = pd.to_datetime(df_hist['Year'], format="%Y") + MonthEnd(12)

In [ ]:
df_hist.head()

In [ ]:
# get a list of economies

economies = df_hist.index.unique()
economies

#### Prepare future data

In [ ]:
df_future_macro = pd.read_csv('../data/raw/Industry/MacroAssumptions.csv',
                              index_col=['Economy'])
df_future_macro['ds'] = pd.to_datetime(df_future_macro['Year'], format="%Y") + MonthEnd(12)
df_future_macro.head()

In [ ]:
df_future_macro.tail()

In [ ]:
# combine the historical and future data

df_big = pd.concat([df_hist,df_future_macro], sort=True)

#### Split data in to training and prediciton sets:

In [ ]:
# update to have a training dataset (instead of 1990-2016)

df_train = df_big[df_big['Year']<=2016]

In [ ]:
# check that the last year is correct

df_train.loc['01_AUS'].tail()

In [ ]:
# make a dataset from 1990-2050 for predictions

df_predict = df_big.drop(columns=targets).drop(columns='Year')

In [ ]:
# check that it worked

df_predict.loc['01_AUS'].head()

In [ ]:
df_predict.loc['01_AUS'].tail()

## 2. Construct the models

#### Construct models by economy and fuel:

In [ ]:
models = {}

for economy in economies:
    nested_dict = {}
    for target in targets:
        m = Prophet(daily_seasonality=False,
                       weekly_seasonality=False,
                       yearly_seasonality=False,
                       seasonality_mode='additive',
                       growth='linear')
        
        # you can add more regressors:
        m.add_regressor('GDP')
        m.add_regressor('Population')
        nested_dict[target] = m
    models[economy] = nested_dict

In [ ]:
# check that individual models are stored in memory
# each economy, fuel model should have its own unique number (e.g., 0x25c9b5261c8)
models

In [ ]:
# dataframes for models

economy_target_dfs = {}

for economy in economies:
    target_dfs = {}
    _df = df_train.loc[economy]
    for target in targets_tuple:
        _df2 = _df[[target,'GDP','Population','ds']]
        _df2 = _df2.rename(columns={target: "y"})
        target_dfs[target] = _df2
    economy_target_dfs[economy] = target_dfs

## 3. Fit the models

In [ ]:
for economy,m1 in models.items():
    print('- The economy is %s' %economy)
    _data_economy = economy_target_dfs[economy]
    _model = m1
    for target,m2 in _model.items():
        _data_target = _data_economy[target]
        model = m2
        print('-- Fitting model for %s' %target)
        model.fit(_data_target)
        
print('\n Finished fitting models')

## 4. Make predictions

In [ ]:
# this part will take awhile to finish

economy_predictions = {}

for economy,m1 in models.items():
    target_predictions = {}
    print('\n- Making prediction for %s' %economy)
    _predict_economy = df_predict.loc[economy]
    _model = m1
    for target,m2 in _model.items():
        model = m2
        forecast = m2.predict(_predict_economy)
        target_predictions[target] = forecast
        print('-- Predicting demand for %s' %target)
    economy_predictions[economy] = target_predictions
    
print('\n Finished making predictions.')

## 5. Extract results

In [ ]:
# combine the results in to a dataframe
# Note that Prophet produces many outputs such as the errors. Only the demand predictions are included right now,
# but you can add those by adding more columns to 'results'

results_list = []

for economy in economies:
    a = economy_predictions[economy]
    for target in targets_tuple:
        b = a[target]
        b['Fuel'] = target
        b['Economy'] = economy
        b['Year'] = b['ds'].dt.year
        b = b.set_index(['Economy','Year','Fuel'])
        _b = b['yhat']
        results_list.append(_b)

_results = pd.concat(results_list)
results = pd.DataFrame(_results)
results.columns = ['Demand']

# this makes sure there are no negative values:
results['Demand'] = np.where(results['Demand'] < 0, 0,results['Demand'])

In [ ]:
# check that the results are in a dataframe

results.tail()

In [ ]:
# write the results to a CSV file (this is in tidy format)

results.to_csv('../data/final/agriculture_results.csv', header=True)

In [ ]:
# this is in regular table format

results_table = results.pivot_table(index=['Economy','Year'],columns='Fuel',values = 'Demand')
results_table.to_csv('../data/final/agriculture_results_table_format.csv', header=True)